In [1]:
import cv2
import numpy as np
import glob
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense
from keras.optimizers import Adam
from keras.models import Sequential
from tensordash.tensordash import Tensordash
import tensorflow as tf

Using TensorFlow backend.
/home/sagnik106/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sagnik106/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sagnik106/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sagnik106/.local/lib/python3.6/site-packages/tenso

In [2]:
key=['melanoma','nevus','seborrheic_keratosis']
x=list()
y=list()
for j in key:
    l=glob.glob('dataset/train/%s_new/*.jpg'%(j))
    c=1
    for i in l:
        a=cv2.imread(i,1)
        a=cv2.resize(a,(128,128))
        x.append(a)
        y.append(j)
        print("%s : %d"%(j,c), end='\r')
        c+=1
    print('\n')
x=np.asarray(x)
y0=np.asarray(y)

melanoma : 2900

nevus : 3660

seborrheic_keratosis : 2790



In [3]:
key=['melanoma','nevus','seborrheic_keratosis']
x0=list()
y1=list()
for j in key:
    l=glob.glob('dataset/train/%s/*.jpg'%(j))
    c=1
    for i in l:
        a=cv2.imread(i,1)
        a=cv2.resize(a,(128,128))
        x0.append(a)
        y1.append(j)
        print("%s : %d"%(j,c), end='\r')
        c+=1
    print('\n')
xval=np.asarray(x0)
yval=np.asarray(y1)

melanoma : 291

nevus : 367

seborrheic_keratosis : 280



In [4]:
enc=OneHotEncoder()
enc.fit(y0.reshape((-1,1)))
y=enc.transform(y0.reshape((-1,1))).toarray()
x=np.float32(x)/255

In [5]:
enc=OneHotEncoder()
enc.fit(yval.reshape((-1,1)))
yval=enc.transform(yval.reshape((-1,1))).toarray()
xval=np.float32(xval)/255

In [11]:
np.save('x.npy',x)
np.save('y.npy',y)

In [6]:
def Model():
    m=Sequential()
    m.add(Conv2D(16,3,input_shape=(128,128,3) ,padding='valid'))
    m.add(MaxPool2D((2,2)))
    m.add(BatchNormalization())
    m.add(Conv2D(32,3, padding='valid'))
    m.add(MaxPool2D((2,2)))
    m.add(BatchNormalization())
    m.add(Conv2D(64,3, padding='valid'))
    m.add(MaxPool2D((2,2)))
    m.add(BatchNormalization())
    m.add(Flatten())
    m.add(Dense(1024, activation='relu'))
    m.add(Dense(3, activation='softmax'))
    return m

In [7]:
model=Model()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 63, 63, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)      

In [8]:
td=Tensordash(email='sagnik106@gmail.com', password='ZeusTrident3',ModelName="C2C")

In [9]:
with tf.device('/gpu:0'):
    model.fit(x,y,epochs=6,batch_size=32, validation_data=(xval,yval), callbacks=[td])


Train on 9350 samples, validate on 938 samples
Epoch 1/6
9350/9350 [==============================] - 94s 10ms/step - loss: 1.1524 - accuracy: 0.6323 - val_loss: 1.5268 - val_accuracy: 0.3987
Epoch 2/6
9350/9350 [==============================] - 92s 10ms/step - loss: 0.5350 - accuracy: 0.7948 - val_loss: 0.7640 - val_accuracy: 0.7228
Epoch 3/6
9350/9350 [==============================] - 95s 10ms/step - loss: 0.2972 - accuracy: 0.8938 - val_loss: 0.6467 - val_accuracy: 0.7569
Epoch 4/6
9350/9350 [==============================] - 97s 10ms/step - loss: 0.1627 - accuracy: 0.9510 - val_loss: 0.6376 - val_accuracy: 0.7814
Epoch 5/6
9350/9350 [==============================] - 101s 11ms/step - loss: 0.0913 - accuracy: 0.9800 - val_loss: 0.6145 - val_accuracy: 0.7964
Epoch 6/6
9350/9350 [==============================] - 98s 10ms/step - loss: 0.0447 - accuracy: 0.9945 - val_loss: 0.6399 - val_accuracy: 0.7793


In [10]:
x.shape

(9350, 128, 128, 3)

In [11]:
model.save('model.h5')